Download and install required pacjages and files.

In [ ]:
!git clone https://github.com/pilichm/VoiceCOntrolledGame.git

In [ ]:
!pip install --upgrade git+git://github.com/pilichm/VoiceCOntrolledGame.git
!pip install git+git://github.com/ricardodeazambuja/colab_utils.git

In [3]:
from pilichm.gameObjects import VoiceModel, GameState, Player
from colab_utils import getAudio
import numpy as np
from scipy.io.wavfile import write
import openfst_python as fst

In [ ]:
!wget https://raw.githubusercontent.com/danijel3/ASRforNLP/main/lexicon.py
from lexicon import words_to_lexicon

In [ ]:
!wget https://github.com/danijel3/ASRforNLP/releases/download/v1.0/kaldi.tar.xz

!tar xvf kaldi.tar.xz -C / > /dev/null
%rm kaldi.tar.xz

!for f in $(find /opt/kaldi -name *.so*) ; do ln -sf $f /usr/local/lib/$(basename $f) ; done
!for f in $(find /opt/kaldi/src -not -name *.so* -type f -executable) ; do ln -s $f /usr/local/bin/$(basename $f) ; done
!for f in $(find /opt/kaldi/tools -not -name *.so* -type f -executable) ; do ln -s $f /usr/local/bin/$(basename $f) ; done

!ldconfig

In [ ]:
!wget https://github.com/danijel3/ASRforNLP/releases/download/v1.2/models.tar.xz
!tar xvf models.tar.xz > /dev/null
%rm models.tar.xz

In [ ]:
%mkdir grammar
%cd grammar
!ln -s ../phonetisaurus
!ln -s ../online
!ln -s ../nagrania

Create model.

In [ ]:
def create_model():
  wordlist=VoiceModel.wordlist
  psyms,wsyms,L=words_to_lexicon(wordlist)
  
  model = VoiceModel.VoiceModel(psyms, wsyms, L)
  fst.determinize(fst.compose(model.l, model.grammar)).minimize().write('LG.fst')
  
  !fstcomposecontext --context-size=2 --central-position=1 --read-disambig-syms=disambig.int --write-disambig-syms=disambig_ilabels.int ilabels LG.fst CLG.fst
  !make-h-transducer --disambig-syms-out=disambig_tid.int ilabels online/tree online/final.mdl H.fst
  !fsttablecompose H.fst CLG.fst - | fstdeterminizestar --use-log=true - - | fstrmsymbols disambig_tid.int - - | fstminimizeencoded - - | add-self-loops --self-loop-scale=0.1 --reorder=true online/final.mdl - HCLG.fst
  
  VoiceModel.change_model_frequency()
  VoiceModel.create_info_about_recording()

create_model()

Function for getting user audio input and extracting action to perform.

In [26]:
def get_direction():
  data, freq = getAudio()
  write('/content/VoiceCOntrolledGame/pilichm/data/recordings/Nagranie.wav', freq, data)
  temp = !online2-wav-nnet3-latgen-faster --word-symbol-table=words.txt  --config=online/conf/online.conf online/final.mdl HCLG.fst ark:spk2utt scp:wav.scp ark:lat
  result = !lattice-best-path ark:lat ark,t:- | /opt/kaldi/egs/wsj/s5/utils/int2sym.pl -f 2- words.txt 
  %rm /content/VoiceCOntrolledGame/pilichm/data/recordings/Nagranie.wav

  cost = VoiceModel.check_cost(result)
  print(f'Cost >{cost}<')
  if cost < 200:
    return VoiceModel.get_direction_from_prediction(result)
  else:
    return VoiceModel.Direction.UNKNOWN
  

Create initial game state.

In [19]:
gs = GameState.GameState()
direction = VoiceModel.Direction.UNKNOWN

Run code below, to record your action.
Available actions:
* 'W lewo' - move left,
* 'W prawo' - move right,
* 'W górę' - move up,
* 'W dół' - move down.

In [ ]:
direction = get_direction()
print(f'{direction}')

Run code below to display current game state.

In [ ]:
gs.perform_player_action(direction)
direction = VoiceModel.Direction.UNKNOWN
if gs.has_ended:
  gs.display_end_screen()
else:
  gs.display_screen()